In [69]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import math
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import statistics as stats
import scipy
from scipy.stats.stats import pearsonr
from matplotlib import style
import warnings
import seaborn as sns
import requests
import bs4 as bs
import os
import csv
warnings.filterwarnings('ignore')

In [150]:
def betaMonthly(ticker):
    '''Finds the 5 year monthly beta of a stock based on the ticker you plan to use.
    This is somewhat inaccurate right now because I just took the average trading days
    in a month for the monthly return instead of taking the actual monthly return of all
    the months depending on how many days are in each month'''
    #start date about 5 years ago based on trading days plus another 21 for calculating monthly returns
    #an extra 21 days because the average amount of trading days in a month is 21 (1260+21)
    start = dt.datetime.now() - dt.timedelta(days=1281)
    end = dt.datetime.now()
    #compare our equity to the market (S&P500)
    market = 'SPY'
    #dataframe for the stock
    r = web.DataReader(ticker,'yahoo',start,end)
    #dataframe for the market
    r1 = web.DataReader(market,'yahoo',start,end)
    #adding a column to easily locate the data for which security
    r['Ticker'] = ticker
    #creating a column labeling it's the market
    r1['Market'] = market
    r1['Market Adj Close'] = r1['Adj Close']
    #setting the columns for each dataframe
    df = r[['Ticker','Adj Close']]
    df1 = r1[['Market','Market Adj Close']]
    stock = df['Adj Close']
    market = df1['Market Adj Close']
    #combining the two dataframes in a way that is easy to read (thats why I used axis)
    dfCombined = pd.concat([df1,df], sort=False, axis=1)
    #average trading days in a month is 21
    monthReturnStock = stock.pct_change(21)
    monthReturnMarket = market.pct_change(21)
    stock1 = monthReturnStock.dropna()
    market1 = monthReturnMarket.dropna()
    #covariance
    covariance = np.cov(stock1,market1)[0][1]
    variance = np.var(market1)
    beta = covariance/variance
    #CHECK
    #correlation for past 3 years
    correlation, pValue = pearsonr(stock1, market1)
    #standard deviation of returns for stock in the past 5 years
    stdevStock = stats.stdev(stock1)
    #standard deviation of returns for market in the past 5 years
    stdevMarket = stats.stdev(market1)
    #formula
    Beta = correlation * (stdevStock/stdevMarket)
    diff = abs(Beta - beta)
    if diff == 0:
        print(Beta)
    elif diff < 1:
        print(beta)
    else:
        print(f'Something is wrong with calculations. This is the first method beta: {beta} and this is the second method: {Beta}') 

In [151]:
#5yr monthly beta of stock of your choice
betaMonthly('TSLA')

1.5571761918677747


In [152]:
def betaWeekly(ticker):
    '''Finds the 5 year weekly beta of a stock based on the ticker you plan to use.
    According to a study done by MIRELA MOMCILOVIC, SANJA VLAOVIC BEGOVIC, STEVAN TOMASEVIC 
    showed that the highest correlation between monthly, weekly, and daily stock betas are
    weekly which means the weekly beta is the most stable.'''
    #start date about 5 years ago based on trading days plus another 5 for calculating weekly returns
    #an extra 5 days because the average amount of trading days in a week is 5 (1260+5)
    start = dt.datetime.now() - dt.timedelta(days=1265)
    end = dt.datetime.now()
    #compare our equity to the market (S&P500)
    market = 'SPY'
    #dataframe for the stock
    r = web.DataReader(ticker,'yahoo',start,end)
    #dataframe for the market
    r1 = web.DataReader(market,'yahoo',start,end)
    #adding a column to easily locate the data for which security
    r['Ticker'] = ticker
    #creating a column labeling it's the market
    r1['Market'] = market
    r1['Market Adj Close'] = r1['Adj Close']
    #setting the columns for each dataframe
    df = r[['Ticker','Adj Close']]
    df1 = r1[['Market','Market Adj Close']]
    stock = df['Adj Close']
    market = df1['Market Adj Close']
    #combining the two dataframes in a way that is easy to read (thats why I used axis)
    dfCombined = pd.concat([df1,df], sort=False, axis=1)
    #average trading days in a week is 5
    weekReturnStock = stock.pct_change(5)
    weekReturnMarket = market.pct_change(5)
    stock1 = weekReturnStock.dropna()
    market1 = weekReturnMarket.dropna()
    #covariance
    covariance = np.cov(stock1,market1)[0][1]
    variance = np.var(market1)
    beta = covariance/variance
    #CHECK
    #correlation for past 5 years
    correlation, pValue = pearsonr(stock1, market1)
    #standard deviation of returns for stock in the past 5 years
    stdevStock = stats.stdev(stock1)
    #standard deviation of returns for market in the past 5 years
    stdevMarket = stats.stdev(market1)
    #formula
    Beta = correlation * (stdevStock/stdevMarket)
    diff = abs(Beta - beta)
    if diff == 0:
        print(Beta)
    elif diff < 1:
        print(beta)
    else:
        print(f'Something is wrong with calculations. This is the first method beta: {beta} and this is the second method: {Beta}')

In [154]:
betaWeekly('TSLA')

1.6879972986055878


In [144]:
def betaDaily(ticker):
    '''Finds the 5 year beta of a stock based on the ticker you plan to use. This is used
    by calculating the daily returns when usually they use monthly returns'''
    #start date about 5 years ago based on trading days plus 1 for calculating daily returns
    #an extra day because I'm going to drop the NaN value (1260+1)
    start = dt.datetime.now() - dt.timedelta(days=1261)
    end = dt.datetime.now()
    #compare our equity to the market (S&P500)
    market = 'SPY'
    #dataframe for the stock
    r = web.DataReader(ticker,'yahoo',start,end)
    #dataframe for the market
    r1 = web.DataReader(market,'yahoo',start,end)
    #adding a column to easily locate the data for which security
    r['Ticker'] = ticker
    #creating a column labeling it's the market
    r1['Market'] = market
    r1['Market Adj Close'] = r1['Adj Close']
    #setting the columns for each dataframe
    df = r[['Ticker','Adj Close']]
    df1 = r1[['Market','Market Adj Close']]
    stock = df['Adj Close']
    market = df1['Market Adj Close']
    #combining the two dataframes in a way that is easy to read (thats why I used axis)
    dfCombined = pd.concat([df1,df], sort=False, axis=1)
    returnStock = stock.pct_change()
    returnMarket = market.pct_change()
    stock1 = returnStock.dropna()
    market1 = returnMarket.dropna()
    #covariance
    covariance = np.cov(stock1,market1)[0][1]
    variance = np.var(market1)
    beta = covariance/variance
    #CHECK
    #correlation for past 5 years
    correlation, pValue = pearsonr(stock1, market1)
    #standard deviation of returns for stock in the past 5 years
    stdevStock = stats.stdev(stock1)
    #standard deviation of returns for market in the past 5 years
    stdevMarket = stats.stdev(market1)
    #formula
    Beta = correlation * (stdevStock/stdevMarket)
    diff = abs(Beta - beta)
    if diff == 0:
        print(Beta)
    elif diff < 1:
        print(beta)
    else:
        print(f'Something is wrong with calculations. This is the first method beta: {beta} and this is the second method: {Beta}')

In [155]:
betaDaily('TSLA')

1.1846586158321297
